In [19]:
import cv2
import os

target_size = (48, 48)

dataset1_path = "FACE/train/surprise"

dataset2_path = "FACE/extra/surprise"


output_path = "FACE/new/surprise"

os.makedirs(output_path, exist_ok=True)
os.makedirs(os.path.join(output_path, "grayscale"), exist_ok=True)
os.makedirs(os.path.join(output_path, "color"), exist_ok=True)

def resize_and_save(input_path, output_path):
    for filename in os.listdir(input_path):
        img_path = os.path.join(input_path, filename)
        img = cv2.imread(img_path)

        
        if len(img.shape) == 2:
            # Grayscale image
            resized_img = cv2.resize(img, target_size)
            cv2.imwrite(os.path.join(output_path, "grayscale", filename), resized_img)
        elif len(img.shape) == 3:
            # Color image
            resized_img = cv2.resize(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), target_size)
            cv2.imwrite(os.path.join(output_path, "color", filename), resized_img)
        else:
            print(f"Skipping unknown image format: {img_path}")

resize_and_save(dataset1_path, output_path)
resize_and_save(dataset2_path, output_path)


In [3]:

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D,BatchNormalization
import numpy as np
import os
from tensorflow.keras.preprocessing import image
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split



x = []
y = []


labels = {"Neutral":0,"Happy":1,"Sad":2,"Surprise":3,"Angry":4}    #dictionary
for label in labels:
	print(label)
	for filename in os.listdir("FACE/fer2013_and_affectnet/"+label):
		if filename.split(".").pop().lower() == "jpg" or filename.split(".").pop().lower() == "png" or filename.split(".").pop().lower() == "jpeg":
			img = image.load_img("FACE/fer2013_and_affectnet/"+label+"/"+filename, target_size=(48,48,1), grayscale=True)
			img = image.img_to_array(img)
			img = img/255.0
			x.append(img)
			y.append(labels[label])


Neutral
Happy
Sad
Surprise
Angry


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, shuffle= True)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)


In [5]:
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(48,48,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Dropout(0.3))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, (3, 3)))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  

model.add(Dense(128))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Dense(64))
model.add(Activation('relu'))
model.add(BatchNormalization())

model.add(Dense(5))
model.add(Activation('softmax'))


In [ ]:
model.compile(loss='categorical_crossentropy',
            optimizer='adam',
            metrics=['accuracy'])

In [7]:
 model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 64)        640       
                                                                 
 activation (Activation)     (None, 46, 46, 64)        0         
                                                                 
 batch_normalization (Batch  (None, 46, 46, 64)        256       
 Normalization)                                                  
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 128)       73856     
                                                                 
 dropout (Dropout)           (None, 21, 21, 128)       0

In [21]:
model.fit(x_train, y_train, epochs=50, batch_size=32, validation_data=(x_test, y_test))
scores = model.evaluate(x_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
model.save('FACE/expression_1.model')

Epoch 1/50
1269/1269 [==============================] - 803s 623ms/step - loss: 1.2453 - accuracy: 0.4864 - val_loss: 1.0417 - val_accuracy: 0.5755
Epoch 2/50
1269/1269 [==============================] - 782s 617ms/step - loss: 0.9297 - accuracy: 0.6322 - val_loss: 1.0909 - val_accuracy: 0.5516
Epoch 3/50
1269/1269 [==============================] - 785s 619ms/step - loss: 0.8322 - accuracy: 0.6764 - val_loss: 0.8726 - val_accuracy: 0.6743
Epoch 4/50
1269/1269 [==============================] - 733s 578ms/step - loss: 0.7675 - accuracy: 0.7034 - val_loss: 0.9859 - val_accuracy: 0.6084
Epoch 5/50
1269/1269 [==============================] - 569s 449ms/step - loss: 0.7066 - accuracy: 0.7284 - val_loss: 0.8332 - val_accuracy: 0.6720
Epoch 6/50
1269/1269 [==============================] - 546s 430ms/step - loss: 0.6529 - accuracy: 0.7497 - val_loss: 0.7613 - val_accuracy: 0.7077
Epoch 7/50
1269/1269 [==============================] - 558s 440ms/step - loss: 0.5847 - accuracy: 0.7770 - val_

INFO:tensorflow:Assets written to: FACE/expression_1.model\assets


INFO:tensorflow:Assets written to: FACE/expression_1.model\assets


In [1]:
#################CORRECT CDOE


import numpy as np
import cv2
import tensorflow as tf
import requests

face_detection = cv2.CascadeClassifier('FACE/haarcascade_files/haar_cascade_face_detection.xml')

model = tf.keras.models.load_model('FACE/expression_1.model')

image_path = 'FACE/test2.jpg' 
img = cv2.imread(image_path)

labels = ["Neutral","Happy","Sad","Surprise","Angry"]

def recommend_movies(emotion):
    api_key = '9f7d7fba833ab36a9e0ed88e2ee2c7ca'
    base_url = 'https://api.themoviedb.org/3/discover/movie'

    genre_mapping = {
        "Sad": 18,       # Drama
        "Angry": 28,     # Action
        "Neutral": 53,   # Thriller
        "Happy": 35,     # Comedy
        "Surprise": 9648  # Mystery
    }

    genre_id = genre_mapping.get(emotion,18)  
    print(f"Emotion: {emotion}, Selected Genres: {genre_id}")

    params = {
        'api_key': api_key,
        'language': 'en-US',
        'sort_by': 'popularity.desc',
        'include_adult': 'false',
        'include_video': 'false',
        'with_genres': genre_id
    }

    response = requests.get(base_url, params=params)
    data = response.json()

    movies = []
    for movie in data.get('results', []):
        title = movie.get('title', '')
        rating = movie.get('vote_average', 0.0)
        movies.append(f"{title} ({rating})")
    return movies




if img is not None:
   
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detected_faces = face_detection.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(50, 50))

    for x, y, w, h in detected_faces:
        
        cv2.rectangle(img, (x, y), (x+w, y+h), (245, 135, 66), 2)
        cv2.rectangle(img, (x, y), (x+w//3, y+20), (245, 135, 66), -1)

    
        face = gray[y+5:y+h-5, x+20:x+w-20]
        face = cv2.resize(face, (48, 48))
        face = face / 255.0

       
        predictions = model.predict(np.array([face.reshape((48, 48, 1))])).argmax()
        emotion = labels[predictions]
        print("EMOTION DETECTED:",emotion)
        movie_recommendations = recommend_movies(emotion)
        print("Recommended Movies:")
        for movie in movie_recommendations:
            print(movie)
else:
    print("Error: Couldn't load the image.")






1/1 [==============================] - 0s 319ms/step
EMOTION DETECTED: Surprise
Emotion: Surprise, Selected Genres: 9648
Recommended Movies:
Thanksgiving (6.7)
Five Nights at Freddy's (7.7)
Hell House LLC Origins: The Carmichael Manor (6.3)
Deep Fear (4.9)
Saw X (7.4)
The Nun II (6.8)
Skydog (6.6)
Foe (5.5)
Leave the World Behind (6.5)
Wifelike (7)
The Batman (7.7)
Counterattack (6.9)
Insidious: The Red Door (6.7)
Anatomy of a Fall (7.7)
The Maze Runner (7.2)
The Feast of Amrita (6)
Glass Onion: A Knives Out Mystery (7.1)
X (6.8)
Scooby-Doo! and Krypto, Too! (7.5)
Hypnotic (6.4)
